In [1]:
import numpy as np
from time import time
from driver import Model

In [2]:
myip = Model("bitstream/cnn.bit")

In [3]:
myip.debug=False

In [4]:
# load test data, check accuracy against software implementation
data = np.int32(np.load("test_x.npy"))
gold = np.load("test_y.npy")

In [5]:
myip.setCNNWeights(np.load("CNN_weights.npy"))
myip.setCNNBias(np.load("CNN_bias.npy"))
myip.setDenseWeights(np.load("dense_weights.npy"))
myip.setDenseBias(np.load("dense_bias.npy"))

In [6]:
# sample data
print(data[0,0,:])
# dataset size
print(data.shape[0])

[ -5427  -4580   2832  -1489  -6022 -11662]
736


In [18]:
myip.set_threshold(0.98)
inference_count = 0
threshld_miss_count = 0
confusion_matrix = np.zeros((4,4))

start_time = time()

player1_current_data_index = 0
player1_current_inference_step = 0
player1_predicted = 0
player1_done = False
player1_gold_result = np.argmax(gold[player1_current_data_index])
player1_prediction_above_threshold = False

player2_current_data_index = data.shape[0]//2
player2_current_inference_step = 0
player2_predicted = 0
player2_done = False
player2_gold_result = np.argmax(gold[player2_current_data_index])
player2_prediction_above_threshold = False

while not player1_done and not player2_done:
    # player 1 part
    if not player1_done:
        # do inference
#         print(f"player 1 inferencing at {player1_current_data_index}, {player1_current_inference_step}")
        player1_predicted = myip.inference(data[player1_current_data_index,\
                                                player1_current_inference_step,\
                                                :], \
                                           user_number=0)
        inference_count += 1
        if player1_predicted != -1:
            player1_prediction_above_threshold = True
            player1_gold_result = np.argmax(gold[player1_current_data_index])
            confusion_matrix[player1_gold_result,player1_predicted] += 1
            player1_current_inference_step = data.shape[1]
            
        # prep for next inference
        player1_current_inference_step += 1
        if player1_current_inference_step >= data.shape[1]:
            if not player1_prediction_above_threshold:
                threshld_miss_count += 1
            player1_current_data_index += 1
            player1_current_inference_step = 0
            myip.resetBuffer(user_number=0)
        if player1_current_data_index == data.shape[0]//2:
            print("player1 done")
            player1_done = True
    
    # player 2 part
    if not player2_done:
        # do inference
#         print(f"player 2 inferencing at {player2_current_data_index - data.shape[0]//2}, {player2_current_inference_step}")
        player2_predicted = myip.inference(data[player2_current_data_index,\
                                                player2_current_inference_step,\
                                                :], \
                                           user_number=1)
        inference_count += 1
        if player2_predicted != -1:
            player2_prediction_above_threshold = True
            player2_gold_result = np.argmax(gold[player2_current_data_index])
            confusion_matrix[player2_gold_result,player2_predicted] += 1
            player2_current_inference_step = data.shape[1]
            
        # prep for next inference
        player2_current_inference_step += 1
        if player2_current_inference_step >= data.shape[1]:
            if not player2_prediction_above_threshold:
                threshld_miss_count += 1
            player2_current_data_index += 1
            player2_current_inference_step = 0
            myip.resetBuffer(user_number=1)
        if player2_current_data_index == data.shape[0]:
            print("player2 done")
            player2_done = True


# for i in range(0, data.shape[0]):
    
# #     print("-"*40 + f"Data: {i}" + "-"*40)
#     gold_result = np.argmax(gold[i])
#     prediction_above_threshold = False
    
#     for j in range(0, data.shape[1]):
        
#         # run inference
#         predicted = myip.inference(data[i,j,:])
#         inference_count += 1
#         if predicted != -1:
#             prediction_above_threshold = True
#             break
        
#     if prediction_above_threshold:
#         confusion_matrix[gold_result,predicted] += 1
#     else:
#         print("not confident enough to give prediction")
#         threshld_miss_count += 1
#     myip.resetBuffer()

print(f"total time for {inference_count} inferences = {time() - start_time}")
print(f"Average inference time = {(time() - start_time)/inference_count}")
print(f"Max inference frequency = {inference_count/(time() - start_time)}")
print(f"Miss rate = {threshld_miss_count/data.shape[0]*100}%")

print(confusion_matrix)

player2 done
total time for 12306 inferences = 10.077383518218994
Average inference time = 0.0008190022471201783
Max inference frequency = 1220.957146230013
Miss rate = 0.0%
[[154.   0.   0.   2.]
 [  1. 129.   3.   1.]
 [  0.   8. 221.   4.]
 [  0.   0.   1. 186.]]
